In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Importar los datos

In [ ]:
admin_src_df = pd.read_csv('data/admission_source_id.csv', 
                           na_values=[" Not Available","Not Available","None","Unknown/Invalid","Not Mapped"," Not Mapped"])
admin_type_df = pd.read_csv('data/admission_type_id.csv',
                           na_values = ["Not Available","None","Not Mapped"])
diabetic_df = pd.read_csv('data/diabetic_data.csv',
                          na_values=["Not Available","None","Unknown/Invalid","Not Mapped","?"])
dschrge_disp_df = pd.read_csv('data/discharge_disposition_id.csv',
                             na_values=["Not Available","None","Unknown/Invalid","Not Mapped"])
#id_mapping_df = pd.read_csv('data/IDs_mapping.csv')    #Es la colección de todas las demás tabals de referencia

In [ ]:
diabetic_df

Revisemos qué hay en cada uno de los dataframes

In [ ]:
admin_src_df
#admin_type_df
#dschrge_disp_df
#diabetic_df.head()

Asignemos los índices originales a los ficheros de las tabals de referencia para desnormalizar la tabla _diabetic_ _ _df_

In [ ]:
admin_src_df.set_index("admission_source_id", inplace = True)
admin_type_df.set_index("admission_type_id", inplace = True)
dschrge_disp_df.set_index("discharge_disposition_id", inplace = True)

Cambiemos el nombre de las descripciones de las tablas de referencia

In [ ]:
admin_src_df.rename(columns={"description":"admin_src_desc"}, inplace = True)
admin_type_df.rename(columns={"description":"admin_type_desc"}, inplace = True)
dschrge_disp_df.rename(columns={"description":"dschrge_disp_desc"}, inplace = True)

Ahora sí ejecutemos los joins. Verifiquemos que si las relaciones son en efecto uno a uno

In [ ]:
print(diabetic_df.join(admin_src_df,how="left", on = "admission_source_id").shape)
print(diabetic_df.join(admin_type_df,how="left", on = "admission_type_id").shape)
print(diabetic_df.join(dschrge_disp_df,how="left", on = "discharge_disposition_id").shape)
print(diabetic_df.shape)

In [ ]:
aug_diabetic_df = diabetic_df.join(admin_src_df,how="left", on="admission_source_id")
aug_diabetic_df = aug_diabetic_df.join(admin_type_df,how="left", on = "admission_type_id")
aug_diabetic_df = aug_diabetic_df.join(dschrge_disp_df,how="left", on = "discharge_disposition_id")

Quitemos los id's que ya denormalizamos

In [ ]:
aug_diabetic_df.drop(labels = ["admission_type_id","discharge_disposition_id","admission_source_id"], 
                     axis = 1, 
                     inplace = True)

In [ ]:
aug_diabetic_df

Ahora entendamos la información que tenemos en el _dataset_

In [ ]:
aug_diabetic_df.set_index(["patient_nbr","encounter_id"], inplace = True)

### Cambiar nombres a las columnas

In [ ]:
aug_diabetic_df.columns = aug_diabetic_df.columns.str.lower().str.strip(' ').str.replace(pat="-",repl="_")

In [ ]:
aug_diabetic_df.rename(columns={"diabetesmed":"diabetes_med","a1cresult":"a1c_result"},inplace=True)

### Typecasting

In [ ]:
aug_diabetic_df.select_dtypes(include = np.number).apply(func = lambda x = x.value_counts(), axis = 1)

## Verifiquemos valores nulos de variables importantes

Antes de revisar valores nulos mal codificados, verifiquemos el total de valores nulos ya identificados

In [ ]:
aug_diabetic_df.isna().sum()/aug_diabetic_df.shape[0]

Hasta el momento tenemos algunos _features_ con hasta 94 y 96% de valores nulos. En el caso del nivel de glucosa podríamos eliminarla, pero como es un _feature_ importante para este tipo de problema, el nivel máximo de glucosa, codifiquemos explícitamente el NA en las variables categóricas. En cuánto al peso, este falta por procedimientos administrativos del hospital, ya que muchos no estaba obligados a tenerlas registradas. Por esto eliminemos primero el peso

In [ ]:
aug_diabetic_df_cat_imp = aug_diabetic_df.select_dtypes(exclude=np.number).fillna("faltante")

In [ ]:
aug_diabetic_df_imp = aug_diabetic_df
aug_diabetic_df_imp[aug_diabetic_df_cat_imp.columns] = aug_diabetic_df_cat_imp

En las variables numéricas las imputaremos con la mediana por ahora. Dado que es dependiente de los datos, esto lo haremos en la etapa de modelamiento

In [ ]:
aug_diabetic_df.select_dtypes(np.number).isna().sum()/aug_diabetic_df.shape[0]